# TV Minimization


In this example $Y=AX+N$ simulation enviroment will be created.
In this equation:
- Y is the observation
- A is system geometry parameters
- X is varible (the thing which is observed )
- N is randomly distrubuted zero mean gaussian noise

Size of this vector and matrices are:
$$ Y_{mx1}=A_{mxn}X_{nx1} + N_{mx1} $$


now contiune with code.


## 1. Configuration

### Import libraries


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline 

plt.rcParams['figure.figsize'] = [16, 10]
plt.gray()


### Helper functions

In [ ]:

def normalize(a):
    return a/255

def denormalize(a):
    return a*255

# image to matrix
def i2m(a):
    return np.array(a)/255

# matrix to image
def m2i(a):
    return Image.fromarray(a*255).convert("L")


### Params

In [ ]:
EPS = 0.0000000001 # epsilon
BETA =  0.2 #
LEARNING_RATE = 0.001

## 2. Init

To simulate the problem the following equation is defined

$ Y = X + N $

where:
- X is the real & non-noisy image ( It is the varible which should be estimated )
- N is random gaussion noise
- Y is the observation


In [ ]:
Xreal = i2m(Image.open('test/phantom.png').convert('L'))
pixelSizeX, pixelSizeY = np.shape(Xreal)
noise = np.random.rand(pixelSizeX, pixelSizeY) * 0.2

Y = Xreal + noise

In [ ]:
m2i(Y)

## 3. Cost function

### Define TV

Tv

$
T V_{2 D}(X)=\sum_{i}^{K} \sum_{j}^{L} \sqrt{\left(X_{i, j}-X_{i-1, j}\right)^{2}+\left(X_{i, j}-X_{i, j-1}\right)^{2}}
$

Gradient of tv

$Grad(X)=\frac{2\left(X_{i, j}-X_{i-1, j}\right)+2\left(X_{i j}-X_{i j-1}\right)}{\sqrt{\left(X_{i, j}-X_{i-1, j}\right)^{2}+\left(X_{i, j}-X_{i, j-1}\right)^{2}+\varepsilon}}
-\frac{2\left(X_{i+1, j}-X_{i, j}\right)}{\sqrt{\left(X_{i+1, j}-X_{i, j}\right)^{2}+\left(X_{i+1, j}-X_{i+1, j-1}\right)^{2}+\varepsilon}}
-\frac{2\left(X_{i, j+1}-X_{i, j}\right)}{\sqrt{\left(X_{i, j+1}-X_{i, j}\right)^{2}+\left(X_{i, j+1}-X_{i-1, j+1}\right)^{2}+\varepsilon}}
$

In [ ]:
def tvNorm(x):
    """Computes the total variation norm and its gradient. From jcjohnson/cnn-vis."""
    x_diff = x - np.roll(x, -1, axis=1)
    y_diff = x - np.roll(x, -1, axis=0)
    grad_norm2 = x_diff**2 + y_diff**2 + EPS
    norm = np.sum(np.sqrt(grad_norm2))
    return norm

def tvGrad(x):
    """Computes the total variation norm and its gradient. From jcjohnson/cnn-vis."""
    x_diff = x - np.roll(x, -1, axis=1)
    y_diff = x - np.roll(x, -1, axis=0)
    grad_norm2 = x_diff**2 + y_diff**2 + EPS
    dgrad_norm = 0.5 / np.sqrt(grad_norm2)
    dx_diff = 2 * x_diff * dgrad_norm
    dy_diff = 2 * y_diff * dgrad_norm
    grad = dx_diff + dy_diff
    grad[:, 1:] -= dx_diff[:, :-1]
    grad[1:, :] -= dy_diff[:-1, :]
    return grad


def l2Norm(x):
    """Computes 1/2 the square of the L2-norm and its gradient."""
    return np.sum(x**2) / 2

def l2NormGrad(x):
    return x


In [ ]:
plt.rcParams['figure.figsize'] = [16, 10]

fig = plt.figure()

ax = fig.add_subplot(121)
ax.title.set_text('Original Image Variation on X Axis')
ax.imshow(m2i(Xreal-np.roll(Xreal, -1, axis=1)))

ax = fig.add_subplot(122)
ax.title.set_text('Noisy Image Variation on X Axis')
ax.imshow(m2i(Y-np.roll(Y, -1, axis=1)))

plt.show()

In [ ]:
testMatrix =  np.arange(9).reshape(3,3)
tvGrad(testMatrix)
#testMatrix

In [ ]:
plt.rcParams['figure.figsize'] = [16, 10]

fig = plt.figure()

ax = fig.add_subplot(121)
ax.title.set_text('Original Image')
ax.imshow(m2i(tvGrad(Xreal)))

ax = fig.add_subplot(122)
ax.title.set_text('Noisy Image')
ax.imshow(m2i(tvGrad(Y)))

plt.show()

### Define Cost Function

$\hat{X}=\underset{X}{\arg \min }\left[\|Y- X\|_{2}+\beta T V(X)\right]$

In [ ]:
def tvCost(Y, X, beta):
    return l2Norm(Y-X) + beta * tvNorm(X)

def tvCostGrad(Y, X, beta):
    return l2NormGrad(Y-X) + beta * tvGrad(X)

In [ ]:
X0 = np.zeros((pixelSizeX, pixelSizeY))
tvNorm(Y), tvGrad(Y)

## 4. Minimization

### Gradient Descent

Generic form of gradient descent.

$X^{k+1} = X^k - \alpha \nabla Cost$

Gradient descent method for tv

$X^{k+1} = X^k - \alpha \nabla( |Y- X\|_{2}+\beta T V(X) ) $


In [ ]:
BETA =  0.3 #
LEARNING_RATE = 0.001

Xk = Y#np.random.rand(pixelSizeX, pixelSizeY)

for i in range(1000):
    if i%10 == 0:
        print(i, tvCost(Y, Xk, BETA))
        m2i(Xk)
    Xnext = Xk - LEARNING_RATE * tvCostGrad(Y, Xk, BETA)
    if tvCost(Y, Xk, BETA) < tvCost(Y, Xnext, BETA):
        break
    
    # init for next iteration
    Xk = Xnext


m2i(Xk)

### Sum all up

In [ ]:
def denoiseTV(X0, beta=BETA, maxIter=1000):
    LEARNING_RATE = 0.001

    Xk = X0

    for i in range(maxIter):
        if i%10 == 0:
            pass
            #print(i, tvCost(Y, Xk, beta))
            #m2i(Xk)
        Xnext = Xk - LEARNING_RATE * tvCostGrad(Y, Xk, BETA)
        # init for next iteration
        if tvCost(Y, Xk, beta) < tvCost(Y, Xnext, beta):
            break
        Xk = Xnext
    return Xnext

m2i(  denoiseTV(Y, 0.9, 90)  )

## 5. Results

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(231)
ax.title.set_text('Original Image')
ax.imshow(m2i(Xreal))

ax = fig.add_subplot(232)
ax.title.set_text('Noisy Image')
ax.imshow(m2i(Y))

ax = fig.add_subplot(233)
ax.title.set_text('TV B=0.3')
ax.imshow(m2i(denoiseTV(Y, 0.3)))

ax = fig.add_subplot(234)
ax.title.set_text('TV B=0.7')
ax.imshow(m2i(denoiseTV(Y, 0.7)))

ax = fig.add_subplot(235)
ax.title.set_text('TV B=0.9')
ax.imshow(m2i(denoiseTV(Y, 0.9)))

ax = fig.add_subplot(236)
ax.title.set_text('TV B=1.1')
ax.imshow(m2i(denoiseTV(Y, 1.1)))

plt.show()
plt.rcParams['figure.figsize'] = [16, 10]
